In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("hugging_face_key")

In [2]:
from huggingface_hub import login
from huggingface_hub import whoami

login(api_key)
whoami()

{'type': 'user',
 'id': '66d5147ab005ad82ca47182f',
 'name': 'dorukozar',
 'fullname': 'Doruk Ozar',
 'email': 'dorukozar@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'periodEnd': None,
 'isPro': False,
 'avatarUrl': '/avatars/06335824f9a6991ec7b901b31802dd5b.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'Presentation',
   'role': 'read',
   'createdAt': '2025-01-16T00:00:59.134Z'}}}

# Reading the file

In [3]:
file_path = 'Week7-lecture.txt'
file_path2 = 'Week9-lecture.txt'

def file_reader(path):
  result = ""

  with open(path, 'r') as file:
      for index, line in enumerate(file):
          if index % 2 == 0:  # Only process even-indexed lines (0-based)
              #print(line.strip())  # Process the line (e.g., print it)
              result += line.strip() + "\n"

  return result

week7 = file_reader(file_path)
week9 = file_reader(file_path2)
# print(week9)

# Splitting the text into Chunks

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(txt):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,  # Define the maximum chunk size
      chunk_overlap=100  # Define the overlap between chunks
  )

  chunks = text_splitter.split_text(txt)
  return chunks

week7_chunks = split_text(week7)
week9_chunks = split_text(week9)

for i, chunk in enumerate(week9_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n{'-'*80}")


Chunk 1:
okay all right so let me pop this up here
bigger i can do that there we go all right so we've been talking about decision trees today all right we'll build a
little bit maybe towards ensemble models towards the end but principally we're just going to talk about decision trees
we're going to talk about how they're made or give you some theoretical background related to uh rates of how they are how constructed
and how they're used they can be used for both regression and classification we're principally going to talk about classification
at this point which is the primary use case for them and then um i'll talk about some of the weaknesses and strengths generally all right so
let's just jump right into it here all right okay so this is what a little
ahead of myself trying to shrink my there we go okay okay
so uh i said all of that already so all right so let's talk right away
--------------------------------------------------------------------------------
Chunk 2:
so uh i said a

# Change the type of text to langchaing document

In [5]:
from langchain.schema import Document
def convert_type(chunks):
  # Convert chunks into LangChain Document objects
  documents = [Document(page_content=chunk) for chunk in chunks]
  return documents

week7_docs = convert_type(week7_chunks)
week9_docs = convert_type(week9_chunks)

# Now `documents` is a list of Document objects ready to be used in a vector store
for i, doc in enumerate(week9_docs):
    print(f"Document {i+1}:\n{doc.page_content}\n{'-'*80}")


Document 1:
okay all right so let me pop this up here
bigger i can do that there we go all right so we've been talking about decision trees today all right we'll build a
little bit maybe towards ensemble models towards the end but principally we're just going to talk about decision trees
we're going to talk about how they're made or give you some theoretical background related to uh rates of how they are how constructed
and how they're used they can be used for both regression and classification we're principally going to talk about classification
at this point which is the primary use case for them and then um i'll talk about some of the weaknesses and strengths generally all right so
let's just jump right into it here all right okay so this is what a little
ahead of myself trying to shrink my there we go okay okay
so uh i said all of that already so all right so let's talk right away
--------------------------------------------------------------------------------
Document 2:
so uh i 

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Setting up the Data base

In [7]:
#create index
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key="pcsk_3GGcqq_DGzzsTybqrBrVXqf46Uf8Kr1yPHZjsMNA9q7feBVViscuc9XpbPs259m8WtgBRX")
index_name = "rag-app"


if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud = "aws", region="us-east-1")
    )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

# Delete all the unnecessary element in the Database

In [8]:
index.delete(delete_all=True)

{}

# Add all the chunks

In [9]:


vector_store.add_documents(week7_docs)
vector_store.add_documents(week9_docs)




['dee51c6e-e9cc-4014-a5e8-2c4736c098c8',
 '0fa73d2e-787b-46a3-8e00-896ca9da8ac3',
 '28424573-46f1-469c-a884-7eb12f3e1ecc',
 '35427ad5-9803-48a6-a38c-c90449b65b4d',
 '9f079db6-92aa-45ec-9fae-ba7f6ff2cd60',
 '6b0736a5-ff60-4fae-beda-d00a26dc7afe',
 '6f87ae92-facb-47a1-9bf0-270188371ddf',
 '9ba22c26-d778-4f90-9057-b7894ba4115f',
 'd47c111f-1262-49b6-9dcc-53053b3a098b',
 '188925c4-b65e-4044-b871-4ac8bb8f2907',
 '1a5794fb-acc8-4db6-9282-d6267330d825',
 'b9891712-e183-4873-90a0-94cedf4c8431',
 '96b173f4-7078-4d68-b93e-362749cbbd26',
 '0eb7047a-02eb-492c-943b-1e48a4233ae4',
 'ff2fd068-e755-40e8-8bab-d6b3e9094194',
 '3695a29d-df90-482e-952e-dfd8cfd569be',
 'c25c1770-4b14-4dcd-8057-c435b2869359',
 '4f35ed08-5a50-4b6f-adfe-278de45a06ba',
 '33f10cb9-c63c-4e19-af9b-6ad55c005198',
 'b2caed61-d393-454d-83aa-fde87ea42535',
 'c7990bd1-fab8-4bb1-9bf4-05a50afda5d4',
 '2d58f743-4b91-4e48-bbd6-4f4420e9c773',
 'df618608-24e5-4167-84a5-e76f2f7ec536',
 'c25655dd-35cf-4236-84b5-b123cfe88c63',
 '7668dd01-aa85-

# PDF Reading

In [8]:
import fitz  # PymMuPDF
import os

def extract_pdf_content(file_path):
    # Create folders for images and text files
    images_folder = "images"
    texts_folder = "texts"
    os.makedirs(images_folder, exist_ok=True)
    os.makedirs(texts_folder, exist_ok=True)

    # Open the PDF file
    doc = fitz.open(file_path)

    for page_num in range(len(doc)):
        page = doc[page_num]

        # Extract text
        text = page.get_text()
        text_filename = os.path.join(texts_folder, f"page_{page_num + 1}.txt")
        with open(text_filename, "w", encoding="utf-8") as text_file:
            text_file.write(text)
        print(f"Saved text for page {page_num + 1} to {text_filename}")

        # Extract images
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = os.path.join(images_folder, f"page_{page_num + 1}_img_{img_index + 1}.png")
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)
            print(f"Saved image to {image_filename}")

    doc.close()

# Path to your PDF file
pdf_path = "Clustering_InClass_9.28.21-1.pdf"

extract_pdf_content(pdf_path)

Saved text for page 1 to texts/page_1.txt
Saved image to images/page_1_img_1.png
Saved image to images/page_1_img_2.png
Saved image to images/page_1_img_3.png
Saved image to images/page_1_img_4.png
Saved text for page 2 to texts/page_2.txt
Saved image to images/page_2_img_1.png
Saved text for page 3 to texts/page_3.txt
Saved text for page 4 to texts/page_4.txt
Saved text for page 5 to texts/page_5.txt
Saved text for page 6 to texts/page_6.txt
Saved image to images/page_6_img_1.png
Saved image to images/page_6_img_2.png
Saved image to images/page_6_img_3.png
Saved text for page 7 to texts/page_7.txt
Saved image to images/page_7_img_1.png
Saved text for page 8 to texts/page_8.txt
Saved image to images/page_8_img_1.png
Saved text for page 9 to texts/page_9.txt
Saved image to images/page_9_img_1.png
Saved text for page 10 to texts/page_10.txt
Saved text for page 11 to texts/page_11.txt
Saved image to images/page_11_img_1.png
Saved image to images/page_11_img_2.png
Saved text for page 12 to

In [9]:
folder_path = 'texts'  # Replace with your file path
elements = os.listdir(folder_path)

sorted_filenames = sorted(elements, key=lambda x: int(x.split('_')[1].split('.')[0]))

combined_texts = ""
print("Sorted filenames:")
for filename in sorted_filenames:
    # print("File Name:", filename)
    with open(folder_path+"/"+filename, 'r') as file:
        for line in file:
            combined_texts += line.strip() + "\n"

print(combined_texts)

Sorted filenames:
Machine Learning Overview, EDA and Clustering
Brian Wright
brianwright@virginia.edu
2
3
Given  D :{2,4,10,12,3,11,20,25,30},  and  k=2 clusters
Randomly assign the means:  m1=3, m2=4
K1={2,3}, K2={4,10,12,11,20,25,30}, m1=2.5, m2=16
K1={2,3,4}, K2={10,12,11,20,25,30}, m1=3, m2=18
K1={2,3,4,10}, K2={12,11,20,25,30}, m1=4.75, m2=19.6
K1={2,3,4,10,11,12}, K2={20,25,30}, m1=7, m2=25
K1={2,3,4,10,11,12}, K2={20,25,30}, m1=7, m2=25
Stop, since the clusters and the means found in
all subsequent iterations will be the same.
Example of K-Means
1. What is Machine Learning?
2. What is exploratory data analysis?
3. k-means clustering
– Does Congress vote in patterns?
4. Multi-dimensional k-means clustering
– Are NBA players compensated according to performance?
Outline: Intro to Unsupervised ML
4
• Exploratory data analysis or “EDA” is an approach where the intent is to see
what the data can tell us beyond modeling or hypothesis testing
– Data visualization is one of the 

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Define the maximum chunk size
    chunk_overlap=100  # Define the overlap between chunks
)

chunks = text_splitter.split_text(combined_texts)


for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n{'-'*80}")

Chunk 1:
Machine Learning Overview, EDA and Clustering
Brian Wright
brianwright@virginia.edu
2
3
Given  D :{2,4,10,12,3,11,20,25,30},  and  k=2 clusters
Randomly assign the means:  m1=3, m2=4
K1={2,3}, K2={4,10,12,11,20,25,30}, m1=2.5, m2=16
K1={2,3,4}, K2={10,12,11,20,25,30}, m1=3, m2=18
K1={2,3,4,10}, K2={12,11,20,25,30}, m1=4.75, m2=19.6
K1={2,3,4,10,11,12}, K2={20,25,30}, m1=7, m2=25
K1={2,3,4,10,11,12}, K2={20,25,30}, m1=7, m2=25
Stop, since the clusters and the means found in
all subsequent iterations will be the same.
Example of K-Means
1. What is Machine Learning?
2. What is exploratory data analysis?
3. k-means clustering
– Does Congress vote in patterns?
4. Multi-dimensional k-means clustering
– Are NBA players compensated according to performance?
Outline: Intro to Unsupervised ML
4
• Exploratory data analysis or “EDA” is an approach where the intent is to see
what the data can tell us beyond modeling or hypothesis testing
--------------------------------------------

In [12]:
from langchain.schema import Document

# Convert chunks into LangChain Document objects
documents = [Document(page_content=chunk) for chunk in chunks]

# Now `documents` is a list of Document objects ready to be used in a vector store
for i, doc in enumerate(documents):
    print(f"Document {i+1}:\n{doc.page_content}\n{'-'*80}")


Document 1:
Machine Learning Overview, EDA and Clustering
Brian Wright
brianwright@virginia.edu
2
3
Given  D :{2,4,10,12,3,11,20,25,30},  and  k=2 clusters
Randomly assign the means:  m1=3, m2=4
K1={2,3}, K2={4,10,12,11,20,25,30}, m1=2.5, m2=16
K1={2,3,4}, K2={10,12,11,20,25,30}, m1=3, m2=18
K1={2,3,4,10}, K2={12,11,20,25,30}, m1=4.75, m2=19.6
K1={2,3,4,10,11,12}, K2={20,25,30}, m1=7, m2=25
K1={2,3,4,10,11,12}, K2={20,25,30}, m1=7, m2=25
Stop, since the clusters and the means found in
all subsequent iterations will be the same.
Example of K-Means
1. What is Machine Learning?
2. What is exploratory data analysis?
3. k-means clustering
– Does Congress vote in patterns?
4. Multi-dimensional k-means clustering
– Are NBA players compensated according to performance?
Outline: Intro to Unsupervised ML
4
• Exploratory data analysis or “EDA” is an approach where the intent is to see
what the data can tell us beyond modeling or hypothesis testing
-----------------------------------------

In [14]:

vector_store.add_documents(documents)

['269383ef-7440-434b-994c-4f4123d42048',
 '0dacfbf6-9874-498e-a6d5-c89a01fc05c3',
 '7f16d771-4e7c-4ce9-b6fd-91479a133e49',
 'b864a8b9-ec26-4792-a584-1e533c05141a',
 '47bfa2b0-4ac8-4466-9adb-b6c326e2b979',
 '8de97e81-28ca-4226-b16c-c1094e856247',
 'b716cb7b-f8ab-4aa8-adae-ce8737fa3df5',
 'cd3a9670-be6f-4b67-95a3-fe26379d3d07',
 '31e86125-33e2-49e8-80df-a41349ce00ff',
 '2977e402-0ba3-4a67-af0e-f195fb2a900b',
 'db8cece0-1c6b-4d3e-a471-046bca7f7fbd',
 '4ef58e04-5875-4da1-9499-3d8344f883c9',
 'a3fa2941-9b69-464f-af1d-ff6f50f040ed',
 '64deae82-c0e1-425a-a912-b354821b77de',
 '8a6c2040-f6c1-4604-8a97-f6a038621d1f',
 'bac165ba-465e-4d3c-851f-7a7413b922e4',
 '980dc6b9-842e-4b01-8b71-9222d972cf5d',
 'bb40e0e0-59e4-426c-8263-9868e7e9c994',
 '2de0d941-5d3b-4f47-9cb7-b42589eacfbb',
 'cde421aa-de2d-4483-9349-209d8c0cc370',
 '8840a948-d142-4db7-9cc6-f028881efeec']

# Retrieving the relevant docs

In [13]:
query = "What are the common challenges or limitations of decision trees?"
retriever = vector_store.as_retriever(search_kwargs={"k":3})
# retriever.get_relevant_documents(query)
retriever.invoke(query)

[Document(id='c7990bd1-fab8-4bb1-9bf4-05a50afda5d4', metadata={}, page_content="unbalanced data sets a single tree can can be can be can be problematic with data sets that are\nquite unbalanced but again that can be cured by moving to kind of more of an ensemble approach\nbut until we get there we have to understand what one tree is doing before we start building forests all right so let's let's take a look all\nright these mathematical approaches and i have some examples all right okay so decision trees uh use several\ntypes of node splitting criteria all right so these are these are those i've talked about these\nuh this is what it would be if we had you know kind of a continuous data problem we're trying to actually predict the value all right as\ncompared to predict the class and these are the two common ones for classification they're really very\nsimilar all right we're going to take a look at we're actually not going to take a look at this one but we're going to take a look at e

# Prepare prompts template

In [14]:
template = """You are an expert LLM assistant specialized in answering questions related to computer science/data science/machine learning/LLM. Use the retrieved information from RAG and your knowledge to respond accurately and clearly to each question.

Guidelines:
1. Provide concise and informative answers.
2. If the question is beyond the scope of your knowledge or the provided information, state, "I don't know."
3. If the context section (the information that is returned by RAG pipeline) has no information about a part of the question, please express that "The retrieved information did not contain answer to this question" but if you can answer it based on your knowledge please do
4. Use examples where applicable to illustrate your answers.
5. Maintain a professional and helpful tone.

Question: {question}

Context:{context}

Answer:
"""

# Format the Documents

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrieved_docs = retriever.invoke(query)
print(format_docs(retrieved_docs))

unbalanced data sets a single tree can can be can be can be problematic with data sets that are
quite unbalanced but again that can be cured by moving to kind of more of an ensemble approach
but until we get there we have to understand what one tree is doing before we start building forests all right so let's let's take a look all
right these mathematical approaches and i have some examples all right okay so decision trees uh use several
types of node splitting criteria all right so these are these are those i've talked about these
uh this is what it would be if we had you know kind of a continuous data problem we're trying to actually predict the value all right as
compared to predict the class and these are the two common ones for classification they're really very
similar all right we're going to take a look at we're actually not going to take a look at this one but we're going to take a look at each one of these

uh we talked about that a little bit we're gonna talk much more about

In [16]:
prompt = template.format(question = query, context =  format_docs(retrieved_docs))
print(prompt)

You are an expert LLM assistant specialized in answering questions related to computer science/data science/machine learning/LLM. Use the retrieved information from RAG and your knowledge to respond accurately and clearly to each question.

Guidelines:
1. Provide concise and informative answers.
2. If the question is beyond the scope of your knowledge or the provided information, state, "I don't know."
3. If the context section (the information that is returned by RAG pipeline) has no information about a part of the question, please express that "The retrieved information did not contain answer to this question" but if you can answer it based on your knowledge please do
4. Use examples where applicable to illustrate your answers.
5. Maintain a professional and helpful tone.

Question: What are the common challenges or limitations of decision trees?

Context:unbalanced data sets a single tree can can be can be can be problematic with data sets that are
quite unbalanced but again that ca

# Generate an answer
### Setting up LLM

In [17]:
import ollama
desired_model = "deepseek-r1:7b"

response = ollama.chat(model=desired_model, messages=[
    {
        'role':'user',
        'content':prompt,
    },
])


ollama_response = response['message']['content']

print(ollama_response)

<think>
Okay, so I'm trying to figure out the common challenges or limitations of decision trees based on the context provided. Let me read through the context again and pick out the relevant points.

The context mentions unbalanced datasets. It says a single tree might be problematic in such cases because it can't handle the imbalance well. That makes sense because if one class dominates, the tree might not perform well for other classes. The context also talks about using ensemble methods like random forests to overcome this issue once we move beyond single trees.

Another point is about node splitting criteria. It mentions different types but doesn't go into detail on their limitations directly. However, decision trees can get complex with too many splits, leading to overfitting. I know that's a common issue because the algorithm makes locally optimal decisions greedily without considering future splits, which might not be globally optimal.

The context also notes that decision tree